# Import Libraries

In [1]:
import cv2
import numpy as np
import os

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.decomposition import PCA

from sklearn import metrics
from sklearn.model_selection import train_test_split

from skimage.feature import canny 
import skimage
from skimage import data, io
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from skimage import color 

import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

# Preprocessing

## Specify directories

In [2]:
#create lists to save the labels (the name of the shape)
train_dir = '/home/trojan/Desktop/pattern recognition/PB2/Implementation/data/abcde'
save_dir = '/home/trojan/Desktop/pattern recognition/PB1/Implementation'
character_list = ['a', 'b', 'c', 'd', 'e']

## Helper functions

In [3]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

## Augmentation with keras

In [4]:
gen = ImageDataGenerator(featurewise_center=True, rotation_range=10, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.15, zoom_range=0.1, 
                         channel_shift_range=0., horizontal_flip=True, vertical_flip=True)

In [5]:
def augmentation_keras(dir):
    for character in character_list:
        for file_name in os.listdir(os.path.join(dir,character)):
                PATH = os.path.join(dir,character)
                img = os.path.join(dir,character,file_name)
                image = cv2.imread(img, -1)
                image = np.expand_dims(cv2.imread(img), 0)
                #image = image.squeeze()
                #plt.figure()
                #plt.imshow(image)
                aug_iter = gen.flow(image, save_to_dir=PATH, save_prefix='aug-image-' + file_name, save_format='png')
                aug_images = [next(aug_iter)[0].astype(np.uint8) for i in range(30)]
                #plotImages(aug_images)
                
                

In [6]:
# Run this block to start augmentation
augmentation_keras(train_dir)

/home/trojan/.local/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


KeyboardInterrupt: 

In [7]:
seed = 5
num_trees = 200

def preprocess(images, labels):

    dataDim = np.prod(images[0].shape)
    images = np.array(images)
    images = images.reshape(len(images), dataDim)
    images = images.astype('float32')
    images /=255
    labels = np.array(labels)
    
    return images, labels


# function to make classifier
def classify(model, images, labels):

    model.fit(images, labels)
    return model


# Training

In [8]:
best_model = []
val_accuracies = []
models = []
trained_models = []
names = []

In [9]:
def train():
    models.append(('KNN', KNeighborsClassifier(n_neighbors=1)))
    #models.append(('LR', LogisticRegression(random_state=seed, max_iter=1000)))
    #models.append(('LDA', LinearDiscriminantAnalysis()))
    #models.append(('CART', DecisionTreeClassifier(random_state=seed)))
    models.append(('RF', RandomForestClassifier(n_estimators=num_trees, random_state=seed)))
    #models.append(('NB', GaussianNB()))
    models.append(('SVM', SVC(random_state=seed)))
    
    #iterate through each shape
    all_labels, all_images = [],[]
    for character in character_list:
        print('Getting data for: ', character)
        for file_name in os.listdir(os.path.join(train_dir,character)):
            all_images.append(cv2.imread(os.path.join(train_dir,character,file_name), 0))
            #add an integer to the labels list
            all_labels.append(character_list.index(character))

    # train and validation split
    train_images, val_images, train_labels, val_labels = train_test_split(all_images, all_labels, 
                                                                          shuffle=True, stratify=all_labels, 
                                                                          test_size=0.1, random_state=42)

    print('Number of training images: ', len(train_images), '\n')

    # Preprocess (your own function)
    train_images, train_labels = preprocess(train_images, train_labels)
    val_images, val_labels = preprocess(val_images, val_labels)

    for name, model in models:
        
        best_model = model
        #train_images, train_labels = preprocess(train_images, train_labels)
        print (name)
        
        # Make a classifier (your own function)
        model = classify(model, train_images, train_labels)
        trained_models.append(model)

        # Predict the labels from the model (your own code depending the output of the train function)
        pred_labels = model.predict(train_images)

        # Calculate accuracy (Do not erase or modify here)
        pred_acc = np.sum(pred_labels==train_labels)/len(train_labels)*100
        print("Accuracy = {}".format(pred_acc))

        cm = metrics.confusion_matrix(train_labels, pred_labels)
        print(cm, '\n')
        
        # Validation
        print('Number of validation images: ', len(val_images))
    
        pred_val_labels = model.predict(val_images)
        val_acc = np.sum(pred_val_labels==val_labels)/len(val_labels)*100
        print("Val Accuracy = {}".format(val_acc), '\n')
        
        val_accuracies.append(val_acc)
        names.append(name)
        

    
        
        
        


In [10]:
if __name__ == '__main__':
    train()

Getting data for:  a
Getting data for:  b
Getting data for:  c
Getting data for:  d
Getting data for:  e
Number of training images:  4178 

KNN
Accuracy = 62.063188128291046
[[670   4  87   5  68]
 [287 446  39  12  51]
 [125   4 571   1 136]
 [315  24 104 338  55]
 [166  33  60   9 568]] 

Number of validation images:  465
Val Accuracy = 40.43010752688172 

RF
Accuracy = 100.0
[[834   0   0   0   0]
 [  0 835   0   0   0]
 [  0   0 837   0   0]
 [  0   0   0 836   0]
 [  0   0   0   0 836]] 

Number of validation images:  465
Val Accuracy = 49.67741935483871 

SVM
Accuracy = 100.0
[[834   0   0   0   0]
 [  0 835   0   0   0]
 [  0   0 837   0   0]
 [  0   0   0 836   0]
 [  0   0   0   0 836]] 

Number of validation images:  465
Val Accuracy = 29.24731182795699 



In [13]:
# Choose the best model

best_acc = np.max(val_accuracies)
print("Best Validation Accuracy = {}".format(best_acc), '\n')
    
index_best_acc = np.argmax(val_accuracies, axis=None)

best_model_initial = models[index_best_acc]
print(best_model_initial)

best_model = trained_models[index_best_acc]
print(best_model)

best_model_name = names[index_best_acc]   
print("Best Model Is {}".format(best_model_name), '\n')

Best Validation Accuracy = 56.774193548387096 

('RF', RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=5, verbose=0,
                       warm_start=False))
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=

## Save the model

In [13]:
import pickle

filename = "final_model"
model_path = os.path.join(save_dir,filename)
pickle.dump(best_model, open(model_path, 'wb'))

## Load the model

In [14]:
'''Model is provided in submission with report and notebook'''

loaded_model = pickle.load(open(model_path, 'rb'))
print(loaded_model)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')


# Testing

In [ ]:
    """forTA (Do not erase here)
    test_dir = '../ForTA'
    test_labels, test_images = [], []
    for shape in shape_list:
        print('Getting data for: ', shape)
        for file_name in os.listdir(os.path.join(test_dir,shape)):
            test_images.append(cv2.imread(os.path.join(test_dir,shape,file_name), 0))
            #add an integer to the labels list
            test_labels.append(shape_list.index(shape))

    print('Number of test images: ', len(test_images))

    test_images, test_labels = preprocess(test_images, test_labels)
    #pred_labels = model.predict(test_images)
    pred_labels = loaded_model.predict(test_images)
    pred_acc = np.sum(pred_labels==test_labels)/len(test_labels)*100
    print("Test Accuracy = {}".format(pred_acc))
    """
